In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
ls

MyDrive/  Shareddrives/


In [ ]:
cd /gdrive/My Drive/Breed Classification

/gdrive/My Drive/Breed Classification


In [ ]:
ls

SheepFaceImages/


In [ ]:
cd /gdrive/My Drive/Breed Classification/SheepFaceImages

/gdrive/My Drive/Breed Classification/SheepFaceImages


In [ ]:
ls

 Marino/  'Poll Dorset'/   Suffolk/  'White Suffolk'/


In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import morphology


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras.utils import to_categorical

In [ ]:
Marino = "/gdrive/My Drive/Breed Classification/SheepFaceImages/Marino/"
WhiteSuffolk = "/gdrive/My Drive/Breed Classification/SheepFaceImages/White Suffolk"
PollDorset = "/gdrive/My Drive/Breed Classification/SheepFaceImages/Poll Dorset"
Suffolk = "/gdrive/My Drive/Breed Classification/SheepFaceImages/Suffolk"

In [ ]:
ls

 Marino/  'Poll Dorset'/   Suffolk/  'White Suffolk'/


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
os.listdir("/gdrive/My Drive/Breed Classification/SheepFaceImages")

['White Suffolk', 'Marino', 'Suffolk', 'Poll Dorset']

In [ ]:
from tqdm import tqdm 
x = []
y = []

def create_dataset(dirname,breedname):
    for i in tqdm(os.listdir(dirname)):
        path = os.path.join(dirname,i)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img,(150,150))
        except:
            continue
            
        x.append(img)
        y.append(breedname)
    return x,y

In [ ]:
x,y = create_dataset(Marino,"Marino")
x,y = create_dataset(WhiteSuffolk,"White Suffolk")
x,y = create_dataset(PollDorset,"Poll Dorset")
x,y = create_dataset(Suffolk,"Suffolk")

100%|██████████| 420/420 [00:00<00:00, 92521.41it/s]


In [ ]:

BASE_PATH = '/gdrive/My Drive/Breed Classification/SheepFaceImages/'

sheep_data = {}

image_filename_index = 0

for sheep_bread in os.listdir(BASE_PATH):
    print(sheep_bread)
    BREAD_PATH = BASE_PATH + '/' + sheep_bread
    for image_filename in os.listdir(BREAD_PATH):
        image_path = BREAD_PATH + '/' + image_filename
        #read the image
        sheep = io.imread(image_path)
        # convert to grayscale
        sheep = rgb2gray(sheep)
        # resize
        sheep = resize(sheep, (90,78), anti_aliasing=False)
        # reshape
        sheep = sheep.reshape(sheep.shape[0] * sheep.shape[1])
        sheep = np.append(sheep, str(sheep_bread))
        sheep_data[image_filename_index] = sheep
        image_filename_index = image_filename_index + 1

column_names = ['pixel_' + str(i) for i in range(0,7020)]
column_names = np.append(column_names, 'bread')
df = pd.DataFrame.from_dict(sheep_data, orient='index', columns=column_names)
df.to_csv('Sheep_bread_dataset.csv', index=None)

White Suffolk
Marino
Suffolk
Poll Dorset


In [ ]:
df = pd.read_csv('Sheep_bread_dataset.csv')
df.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,pixel_10,pixel_11,pixel_12,pixel_13,pixel_14,pixel_15,pixel_16,pixel_17,pixel_18,pixel_19,pixel_20,pixel_21,pixel_22,pixel_23,pixel_24,pixel_25,pixel_26,pixel_27,pixel_28,pixel_29,pixel_30,pixel_31,pixel_32,pixel_33,pixel_34,pixel_35,pixel_36,pixel_37,pixel_38,pixel_39,...,pixel_6981,pixel_6982,pixel_6983,pixel_6984,pixel_6985,pixel_6986,pixel_6987,pixel_6988,pixel_6989,pixel_6990,pixel_6991,pixel_6992,pixel_6993,pixel_6994,pixel_6995,pixel_6996,pixel_6997,pixel_6998,pixel_6999,pixel_7000,pixel_7001,pixel_7002,pixel_7003,pixel_7004,pixel_7005,pixel_7006,pixel_7007,pixel_7008,pixel_7009,pixel_7010,pixel_7011,pixel_7012,pixel_7013,pixel_7014,pixel_7015,pixel_7016,pixel_7017,pixel_7018,pixel_7019,bread
0,0.535896,0.735112,0.596092,0.715351,0.715874,0.807682,0.770199,0.787922,0.663553,0.718444,0.728510,0.748455,0.739403,0.667627,0.627551,0.599327,0.643348,0.630809,0.629887,0.624375,0.616802,0.678042,0.694066,0.719306,0.709292,0.731343,0.732893,0.712976,0.679483,0.651274,0.596216,0.653288,0.695153,0.679461,0.612222,0.413656,0.417541,0.446761,0.622284,0.655431,...,0.103065,0.119948,0.124053,0.119989,0.105199,0.118388,0.186788,0.148008,0.171106,0.245553,0.270931,0.105258,0.131489,0.126402,0.120498,0.114616,0.136163,0.135126,0.189015,0.163547,0.137152,0.152795,0.142948,0.135061,0.116401,0.138360,0.118698,0.126417,0.132708,0.142523,0.142963,0.140325,0.137094,0.141282,0.138352,0.135821,0.159751,0.164631,0.165047,White Suffolk
1,0.570300,0.476585,0.605670,0.734821,0.650246,0.552283,0.534832,0.517925,0.671847,0.732566,0.796880,0.791019,0.759581,0.712675,0.654974,0.525976,0.467258,0.433663,0.414070,0.419942,0.444364,0.415412,0.404864,0.391868,0.383382,0.368611,0.344514,0.390746,0.398044,0.416465,0.339951,0.295015,0.349885,0.351007,0.380615,0.409417,0.409068,0.361171,0.268731,0.291051,...,0.373156,0.320983,0.448412,0.373967,0.401331,0.393619,0.444556,0.410122,0.405405,0.395322,0.365113,0.390477,0.393238,0.402350,0.364394,0.215797,0.132177,0.058859,0.050885,0.034831,0.055532,0.051303,0.061162,0.095700,0.096604,0.060156,0.056234,0.089633,0.049470,0.100407,0.044557,0.060200,0.059252,0.045347,0.042416,0.038495,0.037504,0.037504,0.037504,White Suffolk
2,0.641420,0.714035,0.798457,0.816017,0.803240,0.816083,0.801344,0.743011,0.691599,0.740172,0.728582,0.819638,0.937994,0.946162,0.934094,0.876283,0.821974,0.791593,0.764850,0.735177,0.710309,0.683956,0.712483,0.715435,0.705204,0.697408,0.745241,0.769597,0.797408,0.826072,0.867314,0.871931,0.861623,0.922248,0.923317,0.936312,0.955118,0.954360,0.949872,0.948336,...,0.812868,0.848133,0.935377,0.966684,0.946128,0.947076,0.936303,0.918612,0.852849,0.828449,0.789255,0.778514,0.803982,0.759995,0.793083,0.733971,0.730267,0.684787,0.667140,0.600648,0.756781,0.661824,0.668415,0.657838,0.653524,0.649515,0.610822,0.602674,0.623273,0.432108,0.460594,0.545724,0.365441,0.452674,0.463186,0.423393,0.442913,0.501868,0.562586,White Suffolk
3,0.974750,0.973001,0.966229,0.961779,0.956834,0.971312,0.944290,0.973988,0.968972,0.965677,0.962104,0.958050,0.948977,0.962021,0.942737,0.915821,0.855482,0.806780,0.772257,0.781069,0.764574,0.770878,0.744375,0.774495,0.857635,0.814812,0.764076,0.715056,0.717071,0.595007,0.594371,0.526152,0.477698,0.430217,0.400415,0.351339,0.275882,0.208421,0.221328,0.219212,...,0.214462,0.220314,0.206611,0.185020,0.171316,0.177242,0.196828,0.211523,0.212493,0.202591,0.232177,0.212394,0.241719,0.234028,0.228342,0.239094,0.216490,0.207696,0.193883,0.188927,0.148720,0.166541,0.243851,0.157576,0.189896,0.257533,0.284896,0.304635,0.277249,0.214831,0.298208,0.412674,0.626073,0.673084,0.574031,0.522705,0.490731,0.483027,0.479847,White Suffolk
4,0.725373,0.700187,0.671680,0.644011,0.597497,0.752769,0.817856,0.778771,0.733757,0.745522,0.771142,0.765129,0.758321,0.721153,0.681099,0.704367,0.692214,0.705227,0.721175,0.742343,0.750969,0.766068,0.782702,0.763696,0.758391,0.747596,0.714153,0.698663,0.693750,0.653630,0.620297,0.617

In [ ]:
X = df.drop(['bread'], axis=1).values

encoder = LabelEncoder()
df['bread'] = encoder.fit_transform(df['bread'])
Y = to_categorical(df['bread'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, shuffle=True, stratify=Y, random_state=0)

In [ ]:
X_train = X_train.reshape(len(X_train),90,78,1)
X_test = X_test.reshape(len(X_test),90,78,1)

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(filters=64, kernel_size=3,strides=(2,1), padding='same', activation='relu', input_shape=(90,78,1)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 45, 78, 64)        640       
_________________________________________________________________
flatten_4 (Flatten)          (None, 224640)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                14377024  
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 132       
Total params: 14,379,876
Trainable params: 14,379,876
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=20)

Epoch 1/20
10/10 [==============================] - 7s 559ms/step - loss: 1.6770 - accuracy: 0.3055
Epoch 2/20
10/10 [==============================] - 6s 556ms/step - loss: 1.1311 - accuracy: 0.5118
Epoch 3/20
10/10 [==============================] - 6s 554ms/step - loss: 0.9243 - accuracy: 0.5860
Epoch 4/20
10/10 [==============================] - 5s 534ms/step - loss: 0.7427 - accuracy: 0.7021
Epoch 5/20
10/10 [==============================] - 6s 546ms/step - loss: 0.6493 - accuracy: 0.7059
Epoch 6/20
10/10 [==============================] - 5s 545ms/step - loss: 0.4813 - accuracy: 0.8434
Epoch 7/20
10/10 [==============================] - 5s 534ms/step - loss: 0.4222 - accuracy: 0.8434
Epoch 8/20
10/10 [==============================] - 5s 532ms/step - loss: 0.4109 - accuracy: 0.8689
Epoch 9/20
10/10 [==============================] - 5s 541ms/step - loss: 0.2756 - accuracy: 0.9403
Epoch 10/20
10/10 [==============================] - 5s 544ms/step - loss: 0.2302 - accuracy: 0.9467

In [ ]:
y_train_pred = model.predict_classes(X_train)
y_test_pred = model.predict_classes(X_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
def normalize_value(predictions):
    normalized = []
    for prediction in predictions:
        result = 0
        for i in range(0,len(prediction)):
            result += i * prediction[i]
        normalized.append(int(result))
    return normalized

normalized_y_train = normalize_value(y_train)
normalized_y_test = normalize_value(y_test)

In [ ]:
train_conf_matrix = np.zeros(16).reshape(4,4)

for index in range(0,len(normalized_y_train)):
    train_value = normalized_y_train[index]
    predicted_train_value = y_train_pred[index]
    train_conf_matrix[train_value][predicted_train_value] += 1
    
test_conf_matrix = np.zeros(16).reshape(4,4)

for index in range(0,len(normalized_y_test)):
    test_value = normalized_y_test[index]
    predicted_test_value = y_test_pred[index]
    test_conf_matrix[test_value][predicted_test_value] += 1

In [ ]:
train_conf_matrix_df = pd.DataFrame(train_conf_matrix)
train_conf_matrix_df.columns = ['Marino', 'Poll Dorset', 'Suffolk' , 'White Suffolk']
train_conf_matrix_df.index = ['Marino', 'Poll Dorset', 'Suffolk' , 'White Suffolk']
train_conf_matrix_df

,Marino,Poll Dorset,Suffolk,White Suffolk
Marino,294.0,0.0,0.0,0.0
Poll Dorset,0.0,294.0,0.0,0.0
Suffolk,0.0,0.0,294.0,0.0
White Suffolk,0.0,0.0,0.0,294.0


In [ ]:
test_conf_matrix_df = pd.DataFrame(test_conf_matrix)
test_conf_matrix_df.columns = ['Marino', 'Poll Dorset', 'Suffolk' , 'White Suffolk']
test_conf_matrix_df.index = ['Marino', 'Poll Dorset', 'Suffolk' , 'White Suffolk']
test_conf_matrix_df

,Marino,Poll Dorset,Suffolk,White Suffolk
Marino,97.0,18.0,0.0,11.0
Poll Dorset,7.0,116.0,2.0,1.0
Suffolk,3.0,0.0,123.0,0.0
White Suffolk,12.0,11.0,0.0,103.0


In [ ]:
print("Accuracy Train : 99.97")
print('Accuracy test : %.3f' %(accuracy_score(normalized_y_test,y_test_pred)) )

Accuracy Train : 99.97
Accuracy test : 0.871
